# Model Training
## Models to train:
- Classifiers:
  - Identify if observed orientation changes caused by head or phone, spit out class (movement) + cause (head vs phone)
  - Identify gesture, spit-out class (gesture performed, which will account for the cause also, e.g. head-turned-left vs phone-moved-right) - **Probs not this one**
- Regression:
  - Provide a actual value for the relative orientation + position of head and phone (may also need a classifier for the gesture)

## Variations:
- Single Model:</br>
  Take image, output gesture</br>
  Can only be called when image available and prior prediction complete, as such need to average other data (IMU)</br>
  Might not be possible to have *cond* tensor (e.g. change tensor used based on some condition), so may need a multiply based on input, e.g. provide 0 if face not found, and previous image given?
- Split Model:</br>
  Take image, get head pose, feed into second model with motion data to get gesture</br>
  Second model called as and when data (IMU or 1st model output) available (keeping prior values for pending data), first model called as and when image available</br>
  Can also be called with data averaged (e.g. only when image available)</br>
  Possibly use conditional tensor, will use different tensor object based on condition (e.g. face is visible in frame)
- Half Model:</br>
  Use model but only for some processing, e.g. just get landmarks (see: agarwal2020realtime & guobing2021headpose), or to get gesture
- Dual Model:</br>
  One model to get gesture from head (images), other to get gesture from phone (imu)</br>
  Simple algo to combine input to determine the gesture
  (e.g. one must be stationary, other moving)</br>
  Can possibly have in one model after joining after training? Maybe Markov Model


1. Classifier:
Cascade Classifier (first check if face and landmarks, then feed into trained model)</br>
RNN will be given sequence of Landmarks and IMU data, will classify to a given motion</br>
Might have issue with no 'stationary' class</br>
one-hot encoding for the class (classes will be gesture and direction)

2. Classifiers and HMM:
Use mocap data to determine delta between frames of phone and head movement (each with 6 DoF).</br>
Basically want to know which direction in each DoF is moving (or non if below threshold / can quantise data to speed-up training and runtime? no FP math).</br>
Model to output values of -1, 0, 1 for each DoF the head and phone (how get 3 possible states?)</br>
Feed in this encoded value to HMM (to also be trained), or can then transfer learn, and bolt on an RNN, tho then need to do sequences again?

Need to worry about whether tools used require region containing face to do their work, and if so how best to obtain region containing face that doesn't care for orientation.


# Loading Data
Preparing the data to use (e.g. read the csv into pandas dataframe)

Need to pre-load the images? Rotate them and resize them?</br>
Probably too large to fit all into memory (can exclude some for testing)

In [2]:
from os import listdir
from random import shuffle
import random
import csv
from pathlib import Path
from tkinter import NUMERIC
import pandas
import itertools as iter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn as skl
from sklearn.model_selection import train_test_split
import numpy as np
import cv2 as cv
import torch
import tensorflow as tf
import tensorflow.keras as keras
# from keras.preprocessing.image import img_to_array
# from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras.applications.mobilenet_v2 import MobileNetV2
# import keras.applications.MobileNetV2 as MobileNetV2
# from keras.applications.resnet50 import ResNet50 # quick, lighweight, and fairly accurate

MOTION_ENCODING = {
    '0|0|1': [0, 0, 1],
    '0|1|0': [0, 1, 0],
    '1|0|0': [1, 0, 0],
    
}

CLASSES = {
    'NO_GESTURE' : 0,
    'POINTING_TRANSLATE_PHONE_TOP_CENTRE' : 1,
    'POINTING_TRANSLATE_PHONE_TOP_RIGHT' : 2,
    'POINTING_TRANSLATE_PHONE_MID_RIGHT' : 3,
    'POINTING_TRANSLATE_PHONE_BOTTOM_RIGHT' : 4,
    'POINTING_TRANSLATE_PHONE_BOTTOM_CENTRE' : 5,
    'POINTING_TRANSLATE_PHONE_BOTTOM_LEFT' : 6,
    'POINTING_TRANSLATE_PHONE_MID_LEFT' : 7,
    'POINTING_TRANSLATE_PHONE_TOP_LEFT' : 8,
    'POINTING_ROTATE_PHONE_TOP_CENTRE' : 9,
    'POINTING_ROTATE_PHONE_TOP_RIGHT' : 10,
    'POINTING_ROTATE_PHONE_MID_RIGHT' : 11,
    'POINTING_ROTATE_PHONE_BOTTOM_RIGHT' : 12,
    'POINTING_ROTATE_PHONE_BOTTOM_CENTRE' : 13,
    'POINTING_ROTATE_PHONE_BOTTOM_LEFT' : 14,
    'POINTING_ROTATE_PHONE_MID_LEFT' : 15,
    'POINTING_ROTATE_PHONE_TOP_LEFT' : 16,
    'POINTING_ROTATE_HEAD_TOP_CENTRE' : 17,
    'POINTING_ROTATE_HEAD_TOP_RIGHT' : 18,
    'POINTING_ROTATE_HEAD_MID_RIGHT' : 19,
    'POINTING_ROTATE_HEAD_BOTTOM_RIGHT' : 20,
    'POINTING_ROTATE_HEAD_BOTTOM_CENTRE' : 21,
    'POINTING_ROTATE_HEAD_BOTTOM_LEFT' : 22,
    'POINTING_ROTATE_HEAD_MID_LEFT' : 23,
    'POINTING_ROTATE_HEAD_TOP_LEFT' : 24,
    'TRANSLATE_PHONE_TOP_CENTRE': 25,
    'TRANSLATE_PHONE_MID_RIGHT': 26,
    'TRANSLATE_PHONE_BOTTOM_CENTRE': 27,
    'TRANSLATE_PHONE_MID_LEFT': 28,
    'CIRCULAR_PHONE_CLOCKWISE': 29,
    'CIRCULAR_PHONE_ANTI_CLOCKWISE': 30,
    'CIRCULAR_HEAD_CLOCKWISE' : 31,
    'CIRCULAR_HEAD_ANTI_CLOCKWISE' : 32,
    'ZOOM_PHONE_ZOOM_IN' : 33,
    'ZOOM_PHONE_ZOOM_OUT' : 34,
    'ZOOM_HEAD_ZOOM_IN' : 35,
    'ZOOM_HEAD_ZOOM_OUT' : 36,
    'ROTATE_HEAD_TOP_CENTRE' : 37,
    'ROTATE_HEAD_MID_RIGHT' : 38,
    'ROTATE_HEAD_BOTTOM_CENTRE' : 39,
    'ROTATE_HEAD_MID_LEFT' : 40,
    'ROTATE_PHONE_ROLL_CLOCKWISE' : 41,
    'ROTATE_PHONE_ROLL_ANTI_CLOCKWISE : 42'
    'ROTATE_HEAD_ROLL_CLOCKWISE' : 43,
    'ROTATE_HEAD_ROLL_ANTI_CLOCKWISE' : 44
}

YUNET_FIELDS = [
    'BB_X','BB_WIDTH','BB_Y','BB_HEIGHT','BB_EYE_R_X','BB_EYE_R_Y','BB_EYE_L_X','BB_EYE_L_Y',
    'BB_NOSE_X','BB_NOSE_Y','BB_MOUTH_R_X','BB_MOUTH_R_Y','BB_MOUTH_L_X','BB_MOUTH_L_Y'
]
MOTION_YUNET_FIELDS = [*[f"{field}_1" for field in YUNET_FIELDS], *[f"{field}_2" for field in YUNET_FIELDS]]

POSE_INPUT_CSV_FIELD_NAMES = [
    'DELTA', 'IMG_PATH', 'FACE_DETECTED', *YUNET_FIELDS, 
    'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z',
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_ROLL', 'PHONE_PITCH', 'PHONE_YAW', 
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_ROLL', 'HEAD_PITCH', 'HEAD_YAW'
]

MOTION_INPUT_CSV_FIELD_NAMES = [
    'DELTA', 'IMG_PATH_1', 'FACE_DETECTED_1', *[f"{field}_1" for field in YUNET_FIELDS], 
    'IMG_PATH_2', 'FACE_DETECTED_2', *[f"{field}_2" for field in YUNET_FIELDS], 
    'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z',
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_ROLL', 'PHONE_PITCH', 'PHONE_YAW', 
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_ROLL', 'HEAD_PITCH', 'HEAD_YAW'
]

POSE_NUMERIC_FIELDS = [
    'DELTA', *YUNET_FIELDS, 'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z',
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_ROLL', 'PHONE_PITCH', 'PHONE_YAW', 
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_ROLL', 'HEAD_PITCH', 'HEAD_YAW'
]

MOTION_NUMERIC_FIELDS = [
    'DELTA', *MOTION_YUNET_FIELDS, 'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z'
]

ACCELERATION_FIELDS = [
    'LINEAR_X', 'LINEAR_Y', 'LINEAR_Z', 'ROTATION_X', 'ROTATION_Y', 'ROTATION_Z'
]

MOTION_ENCODED_FIELDS = [
    'PHONE_X', 'PHONE_Y', 'PHONE_Z', 'PHONE_ROLL', 'PHONE_PITCH', 'PHONE_YAW',
    'HEAD_X', 'HEAD_Y', 'HEAD_Z', 'HEAD_ROLL', 'HEAD_PITCH', 'HEAD_YAW'
]

In [77]:
import time
IMAGE_PATH = Path('/mnt/e/Uni/CM50175/Study/data/synchronisation')
image_width = 180
image_height = 320
image_channels = 3
img_input_shape = (image_height, image_width*2, image_channels)

# Derived from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

def import_img(path):
    img = cv.imread(str(path))
    img = cv.resize(img, (320, 180))
    img = np.rot90(img)
    img = np.ascontiguousarray(img)
    return img

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_frame, batch_size=10, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.data = data_frame
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.data.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.data.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indices):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        img_input = np.empty((self.batch_size, *img_input_shape))
        acc_input = np.empty((self.batch_size, 6))
        y = {field: np.empty((self.batch_size, 3)) for field in MOTION_ENCODED_FIELDS}# np.empty((self.batch_size, 12, 3))

        # Generate data
        for i, idx in enumerate(indices):
            # Store sample
            img_path_1 = IMAGE_PATH / f"{self.data.iloc[idx]['IMG_PATH_1']}.png"
            img_path_2 = IMAGE_PATH / f"{self.data.iloc[idx]['IMG_PATH_2']}.png"

            img_1 = import_img(str(img_path_1))
            img_2 = import_img(str(img_path_2))
            # img_concat = np.concatenate((img_1, img_2), axis=1)
            # print(img_concat.shape)
            # cv.imshow('concat image', img_concat)
            # cv.waitKey(0)
            img_input[i] = np.concatenate((img_1, img_2), axis=1)#img_concat
            acc_input[i] = np.array(self.data.iloc[idx][ACCELERATION_FIELDS].values)
            # print(acc_input)
            # acc_input = np.concatenate(acc_input)

            # Store class
            for field, batch in y.items():
                batch[i] = self.data.iloc[idx][field]
            # y[i] = np.concatenate(self.data.iloc[idx][MOTION_ENCODED_FIELDS].values).reshape((12,3))

        return (img_input, acc_input), (y['PHONE_X'], y['PHONE_Y'], y['PHONE_Z'], y['PHONE_ROLL'], y['PHONE_PITCH'], y['PHONE_YAW'], y['HEAD_X'], y['HEAD_Y'], y['HEAD_Z'], y['HEAD_ROLL'], y['HEAD_PITCH'], y['HEAD_YAW'])


In [22]:
import time

DATA_PATH = Path('/mnt/e/Uni/CM50175/Study/data/training_data/NN/MOTION')

def get_file_paths():
    return [(file_path.name) for file_path in DATA_PATH.iterdir()]

def time_formatted(delta_ms):
    min = int((delta_ms/(1000*60))%60)
    sec = int(((delta_ms - min)/1000)%60)
    ms = int(delta_ms - (min*60*1000) - (sec*1000))
    return min, sec, ms

def convert_encoding_str_to_array(str):
    return np.array([float(val) for val in str.split('|')])
    
def get_data_frame(input_files_root, input_files):
    first = round(time.time() * 1000)
    imported_data = []
    count = 0
    last_batch = first
    for file in input_files:
        count += 1
        participant = file[-7]
        df = pandas.read_csv(input_files_root / file)
        
        # process columns
        df = df.rename(columns={'LINEAR_X.1': 'LINEAR_Z'}) # wrong output resulting in duped column name...
        df[MOTION_NUMERIC_FIELDS] = df[MOTION_NUMERIC_FIELDS].apply(pandas.to_numeric, errors='coerce')
        df[MOTION_ENCODED_FIELDS] = df[MOTION_ENCODED_FIELDS].apply(lambda series: series.map(convert_encoding_str_to_array))
        df[['IMG_PATH_1', 'IMG_PATH_2']] = df[['IMG_PATH_1', 'IMG_PATH_2']].apply(lambda series: series.map(lambda img_path: f"{participant}/{img_path}"))
        df.astype({'FACE_DETECTED_1': bool, 'FACE_DETECTED_2': bool})
        # print(df.keys())
        df = df.drop(MOTION_YUNET_FIELDS, axis=1)

        imported_data.append(df)

        # log summary
        if count % 2000 == 0:
            now = round(time.time() * 1000)
            first_diff = time_formatted(now-first)
            batch_diff = time_formatted(now-last_batch)
            print(f"{count} Files read - {batch_diff[0]}:{batch_diff[1]}.{batch_diff[2]} for current batch, {first_diff[0]}:{first_diff[1]}.{first_diff[2]} since start")
            last_batch = now

    print(f"All {count} Files Loaded")
    concat_now = round(time.time() * 1000)
    imported_data = pandas.concat(imported_data, axis=0, ignore_index=True, sort=False)
    now = round(time.time() * 1000)
    first_diff = time_formatted(now-first)
    concat_diff = time_formatted(now-concat_now)
    print(f"Concat Complete - {concat_diff[0]}:{concat_diff[1]}.{concat_diff[2]} for concat, {first_diff[0]}:{first_diff[1]}.{first_diff[2]} since start")
    return imported_data

file_paths = get_file_paths()
print(f'Number of files to process: {len(file_paths)}')

train_val_paths, test_paths = train_test_split(file_paths, test_size=0.2, random_state=28)
train_paths, val_paths = train_test_split(train_val_paths, test_size=0.2, random_state=16)

print(f"Testing paths: {len(test_paths)}, Training: {len(train_paths)}, Validation: {len(val_paths)}")

train_data_frame = get_data_frame(DATA_PATH, train_paths)
val_data_frame = get_data_frame(DATA_PATH, val_paths)
test_data_frame = get_data_frame(DATA_PATH, test_paths)

Number of files to process: 22632
Testing paths: 4527, Training: 14484, Validation: 3621
2000 Files read - 0:51.532 for current batch, 0:51.532 since start
4000 Files read - 0:51.197 for current batch, 1:42.729 since start
6000 Files read - 0:50.452 for current batch, 2:33.181 since start
8000 Files read - 0:51.529 for current batch, 3:24.710 since start
10000 Files read - 0:50.713 for current batch, 4:15.423 since start
12000 Files read - 0:50.441 for current batch, 5:5.864 since start
14000 Files read - 0:49.273 for current batch, 5:55.137 since start
All 14484 Files Loaded
Concat Complete - 0:4.403 for concat, 6:11.712 since start
2000 Files read - 0:51.898 for current batch, 0:51.898 since start
All 3621 Files Loaded
Concat Complete - 0:1.331 for concat, 1:35.378 since start
2000 Files read - 0:50.899 for current batch, 0:50.899 since start
4000 Files read - 0:49.552 for current batch, 1:40.451 since start
All 4527 Files Loaded
Concat Complete - 0:1.476 for concat, 1:55.260 since s

# Build Model

In [74]:
# Transfer Learning Motion Encoder
regularisation = 0.01
dropout = 0.2

# concat_img = np.concatenate((old_img, new_img), axis=0)
img_input = keras.Input(shape=img_input_shape)
mnetv2 = MobileNetV2(input_tensor=img_input, input_shape=img_input_shape, weights='imagenet', include_top=False)
mnetv2.trainable = False
img_cnn = layers.Conv2D(512, (4,4), padding='same', activation='relu')(mnetv2.output)
img_cnn = layers.Flatten()(img_cnn)
img_fcn = layers.Dense(256, activation='relu', activity_regularizer=tf.keras.regularizers.l2(regularisation))(img_cnn)
img_fcn = layers.Dropout(dropout)(img_fcn)
img_fcn = layers.Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.l2(regularisation))(img_fcn)
img_fcn = layers.Dropout(dropout)(img_fcn)
img_fcn_out = layers.Dense(128, activation='relu', activity_regularizer=tf.keras.regularizers.l2(regularisation))(img_fcn)
# combined_img_model = keras.Model(inputs=[old_img_model.input, new_img_model.input], outputs=img_fcn_out, name='Combined IMG Model')

# The Linear and Angular acceleration from the phone
acc_in = keras.Input(shape=(6,))
acc_fcn = keras.layers.Dense(36, activation='relu')(acc_in)
# acc_model = keras.Model(inputs=input_acc, outputs=acc_fcn, name='ACC Model')

final_model = keras.layers.Concatenate()([img_fcn_out, acc_fcn])
final_model = layers.Dropout(dropout)(final_model)
final_model = keras.layers.Dense(36, activation='relu')(final_model)
final_model = layers.Dropout(dropout)(final_model)

phone_x = keras.layers.Dense(3, name="Phone_x")(final_model)
phone_y = keras.layers.Dense(3, name="Phone_y")(final_model)
phone_z = keras.layers.Dense(3, name="Phone_z")(final_model)
phone_roll = keras.layers.Dense(3, name="Phone_roll")(final_model)
phone_pitch = keras.layers.Dense(3, name="Phone_pitch")(final_model)
phone_yaw = keras.layers.Dense(3, name="Phone_yaw")(final_model)
head_x = keras.layers.Dense(3, name="Head_x")(final_model)
head_y = keras.layers.Dense(3, name="Head_y")(final_model)
head_z = keras.layers.Dense(3, name="Head_z")(final_model)
head_roll = keras.layers.Dense(3, name="Head_roll")(final_model)
head_pitch = keras.layers.Dense(3, name="Head_pitch")(final_model)
head_yaw = keras.layers.Dense(3, name="Head_yaw")(final_model)

tl_model = keras.Model(inputs=[img_input, acc_in], outputs=[phone_x,phone_y,phone_z,phone_roll,phone_pitch,phone_yaw,head_x,head_y,head_z,head_roll,head_pitch,head_yaw], name="ModelA")
tl_model.summary()
plot_model(tl_model, show_shapes=True)

Model: "ModelA"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 320, 360, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 160, 180, 32  864         ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 160, 180, 32  128         ['Conv1[0][0]']                  
                                )                                                            

# Train Model

In [80]:
from keras.metrics import categorical_accuracy
# Code sampled from coursework from another unit: https://colab.research.google.com/drive/1rFKfHzzp0lgjYBCbP1GG0Ttn77acamw2?authuser=2#scrollTo=1pl5bSA5VT3Y
train_ds = DataGenerator(train_data_frame, batch_size=128)
val_ds = DataGenerator(val_data_frame, batch_size=128)
test_ds = DataGenerator(test_data_frame, batch_size=128)

model_path = Path('/mnt/e/Uni/CM50175/Study/data/training_data/NN/')
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001, # learning rate determined by hyper-param-tuning
    decay_steps=(train_ds.__len__() // 32) * 2, # decay every 2 epochs
    decay_rate=0.9)

# compile model
tl_model_compiled = tl_model.compile(loss=keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics='accuracy')

tl_model_epoch_count = 20
tl_model_train_steps_per_epoch = train_ds.__len__() // train_ds.batch_size
tl_model_val_steps_per_epoch = val_ds.__len__() // train_ds.batch_size

print("training steps per epoch: {}\nvalidation steps per epoch: {}".format(tl_model_train_steps_per_epoch, tl_model_val_steps_per_epoch))
checkpoint_path = model_path / 'checkpoints'
checkpoint_path.mkdir(exist_ok=True)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(checkpoint_path),
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5,
    mode='min', restore_best_weights=True
)

print("Fitting Transfer Learning Model (using MobileNetV2)")
tl_model_history = tl_model.fit(
    x = train_ds,
    validation_data=val_ds,
    epochs=tl_model_epoch_count,
    steps_per_epoch=tl_model_train_steps_per_epoch,
    validation_steps=tl_model_val_steps_per_epoch,
    callbacks=[early_stop_cb, model_checkpoint_callback]
)

tl_model.save(str(model_path / "TL_MODEL.h5"))

training steps per epoch: 68
validation steps per epoch: 17
Fitting Transfer Learning Model (using MobileNetV2)
Epoch 1/20


2022-09-07 15:28:54.237411: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1415577600 exceeds 10% of free system memory.
2022-09-07 15:28:54.502057: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1432338432 exceeds 10% of free system memory.


 1/68 [..............................] - ETA: 31:48 - loss: 41.5847 - Phone_x_loss: 6.3170 - Phone_y_loss: 3.1115 - Phone_z_loss: 2.1596 - Phone_roll_loss: 2.3650 - Phone_pitch_loss: 6.5688 - Phone_yaw_loss: 2.7207 - Head_x_loss: 1.9204 - Head_y_loss: 1.3181 - Head_z_loss: 6.1628 - Head_roll_loss: 1.2456 - Head_pitch_loss: 6.3980 - Head_yaw_loss: 1.1196 - Phone_x_accuracy: 0.3984 - Phone_y_accuracy: 0.2812 - Phone_z_accuracy: 0.3047 - Phone_roll_accuracy: 0.5000 - Phone_pitch_accuracy: 0.2188 - Phone_yaw_accuracy: 0.1016 - Head_x_accuracy: 0.2578 - Head_y_accuracy: 0.3828 - Head_z_accuracy: 0.6094 - Head_roll_accuracy: 0.7344 - Head_pitch_accuracy: 0.1797 - Head_yaw_accuracy: 0.6641

2022-09-07 15:29:15.171898: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1415577600 exceeds 10% of free system memory.
2022-09-07 15:29:15.404626: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1432338432 exceeds 10% of free system memory.


 2/68 [..............................] - ETA: 22:34 - loss: 40.6658 - Phone_x_loss: 6.1705 - Phone_y_loss: 2.7611 - Phone_z_loss: 1.7152 - Phone_roll_loss: 2.3161 - Phone_pitch_loss: 6.5327 - Phone_yaw_loss: 2.4820 - Head_x_loss: 1.9185 - Head_y_loss: 1.2993 - Head_z_loss: 6.3257 - Head_roll_loss: 1.2973 - Head_pitch_loss: 6.3903 - Head_yaw_loss: 1.3273 - Phone_x_accuracy: 0.3789 - Phone_y_accuracy: 0.3086 - Phone_z_accuracy: 0.3477 - Phone_roll_accuracy: 0.5000 - Phone_pitch_accuracy: 0.1914 - Phone_yaw_accuracy: 0.0898 - Head_x_accuracy: 0.2812 - Head_y_accuracy: 0.3906 - Head_z_accuracy: 0.5664 - Head_roll_accuracy: 0.7461 - Head_pitch_accuracy: 0.1875 - Head_yaw_accuracy: 0.6445

2022-09-07 15:29:36.311529: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1415577600 exceeds 10% of free system memory.


 4/68 [>.............................] - ETA: 21:54 - loss: 40.2667 - Phone_x_loss: 6.5527 - Phone_y_loss: 2.6186 - Phone_z_loss: 1.6912 - Phone_roll_loss: 2.2836 - Phone_pitch_loss: 5.8722 - Phone_yaw_loss: 2.4984 - Head_x_loss: 1.8260 - Head_y_loss: 1.3671 - Head_z_loss: 6.3019 - Head_roll_loss: 1.1319 - Head_pitch_loss: 6.5330 - Head_yaw_loss: 1.4963 - Phone_x_accuracy: 0.3750 - Phone_y_accuracy: 0.3145 - Phone_z_accuracy: 0.3574 - Phone_roll_accuracy: 0.4688 - Phone_pitch_accuracy: 0.1797 - Phone_yaw_accuracy: 0.0977 - Head_x_accuracy: 0.2656 - Head_y_accuracy: 0.3848 - Head_z_accuracy: 0.5898 - Head_roll_accuracy: 0.7520 - Head_pitch_accuracy: 0.1484 - Head_yaw_accuracy: 0.6523

# Model Evaluation

In [ ]:
# Code sampled from coursework from another unit: https://colab.research.google.com/drive/1rFKfHzzp0lgjYBCbP1GG0Ttn77acamw2?authuser=2#scrollTo=1pl5bSA5VT3Y
epochs = np.linspace(0, tl_model_epoch_count, tl_model_epoch_count)

loss_labels = ['Phone_x_loss', 'Phone_y_loss', 'Phone_z_loss', 'Phone_roll_loss', 'Phone_pitch_loss', 'Phone_yaw_loss', 'Head_x_loss', 'Head_y_loss', 'Head_z_loss', 'Head_roll_loss', 'Head_pitch_loss', 'Head_yaw_loss'] 
accuracy_labels = ['Phone_x_accuracy', 'Phone_y_accuracy', 'Phone_z_accuracy', 'Phone_roll_accuracy', 'Phone_pitch_accuracy', 'Phone_yaw_accuracy', 'Head_x_accuracy', 'Head_y_accuracy', 'Head_z_accuracy', 'Head_roll_accuracy', 'Head_pitch_accuracy', 'Head_yaw_accuracy']

fig, (encoding_loss, encoding_accuracy) = plt.subplots(2, sharex=True)
fig.suptitle("Motion Encoder Learning Curves")
fig.set_size_inches(12,8)

encoding_loss.set_xlabel("Epoch")
encoding_loss.set_ylabel("Loss")
for loss in loss_labels:
    encoding_loss.plot(epochs, tl_model_history.history[loss], label=loss[:-5])
    encoding_loss.plot(epochs, tl_model_history.history[f'{loss}_val'], label=f'{loss[:-5]}_val')

encoding_loss.legend()

encoding_accuracy.set_xlabel("Epoch")
encoding_accuracy.set_ylabel("Accuracy")
for accuracy in accuracy_labels:
    encoding_accuracy.plot(epochs, tl_model_history.history[accuracy], label=accuracy[:-9])
    encoding_accuracy.plot(epochs, tl_model_history.history[f'{accuracy}_val'], label=f'{accuracy[:-9]}_val')
encoding_accuracy.legend()

plt.show()

# Gesture Classification

In [ ]:
# RL - From my CM50270 Coursework 1

import random
import numpy as np

# Setup Constants 
exploration_probability = 15e-2 # epsilon
discount_factor = 9e-1 # gamma
step_size = 2e-1 # alpha
episode_count = 150
agent_count = 20

q_learning_rewards = []

# This needs to represent a series of gestures to classify, do I need more data?, e.g. a sequence of gestures from which it should learn
# can the 'env' change each time?

# Maybe try DQN?

# How to calculate reward? just 'wrong' if not correct, or 
class Env():
    def __init__(self, data_dir):
        pass


class QLAgent():
    def __init__(self):
        self.Q = {} # state -> action -> expected return?

        # internal state should be:
        #   - ([Previous_Gesture_Guesses], Observation) > action > expected return
        # action is guess at a gesture, e.g. what we think the current gesture is
        # Observation is the motion_encoded state from our Neural Network
        # should we use sequence of previous gestures?
        
    def learn(self, episode_count, exploration_probability, discount_factor, step_size, env, run_final_policy=False):
        rewards = []
        for episode in range(0, episode_count):
            undiscounted_returns = self.run_policy_till_terminal(env, exploration_probability, step_size, discount_factor)
            rewards.append(undiscounted_returns)
        
        if run_final_policy:
            self.run_policy_till_terminal(env, exploration_probability, step_size, discount_factor, True)
            
        return rewards, self.Q
        
    def select_action(self, S, available_actions, exploration_probability):
        if S in self.Q:
            policy_actions = self.Q[S]
        else:
            policy_actions = {}
            for available_action in available_actions:
                policy_actions[available_action] = 0
            self.Q[S] = policy_actions # default to save time in action evaluation

        action = None
        explore = random.uniform(0., 1.) < exploration_probability
        if not explore:
            best_return = -np.inf
            for (possible_action, expected_return) in policy_actions.items():
                if expected_return > best_return: 
                    best_return = expected_return
                    action = possible_action
        if action is None:
            action = available_actions[random.randint(0, len(available_actions)-1)]
        return action

    def evaluate_action(self, state, action, reward, state_prime, step_size, discount_factor, available_actions, exploration_probability):
        action_prime = self.select_action(state_prime, available_actions, -1) # best next action given new state
        current_expected_reward = self.Q[state][action]
        self.Q[state][action] = current_expected_reward + (step_size * (reward + (discount_factor * self.Q[state_prime][action_prime]) - current_expected_reward))
        
    def run_policy_till_terminal(self, env, exploration_probability, step_size, discount_factor, render=False):
        undiscounted_G = 0 # running undiscounted return

        # Instantiate environment object.
        state = env.reset()
        available_actions = env.get_actions()
        terminal = False

        while (not terminal):
            if (render):
                env.print()
            action = self.select_action(state, available_actions, exploration_probability) # select action A for given state S
            new_state, reward, terminal = env.step(action)
            undiscounted_G = undiscounted_G + reward # track returns
            self.evaluate_action(state, action, reward, new_state, step_size, discount_factor, available_actions, exploration_probability) # evaluate action A for current state S
            state = new_state

        if (render):
            env.print()
        
        return undiscounted_G

q_learning_agent = QLAgent()
q_learning_agent_rewards = q_learning_agent.learn(episode_count, exploration_probability, discount_factor, step_size, env)[0] #, agent == agent_count-1)[0]
q_learning_rewards.append(q_learning_agent_rewards)